In [47]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, InputLayer
from keras.wrappers.scikit_learn import KerasRegressor
import re

In [166]:
train = pd.read_csv('./datasets/titanic/train.csv')
test = pd.read_csv('./datasets/titanic/test.csv')
num_train = len(train)

In [167]:
df_all = pd.concat([train, test], sort=False)

In [168]:
df_all.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [169]:
df_all['Embarked'] = df_all['Embarked'].fillna('S')

In [170]:
df_all = df_all.drop(columns=['Ticket', 'Cabin'])

age proccesing 

In [171]:
df_all.loc[ df_all['Age'] <= 16, 'Age'] = 0
df_all.loc[(df_all['Age'] > 16) & (df_all['Age'] <= 32), 'Age'] = 1
df_all.loc[(df_all['Age'] > 32) & (df_all['Age'] <= 48), 'Age'] = 2
df_all.loc[(df_all['Age'] > 48) & (df_all['Age'] <= 64), 'Age'] = 3
df_all.loc[ df_all['Age'] > 64, 'Age'] = 4

In [172]:
for i in range(0, 2):
    for j in range(0, 3):
        df_all.loc[(df_all['Age'].isnull()) & (df_all['Sex'] == i) & (df_all['Pclass'] == j + 1), 'Age'] = df_all[(df_all['Sex'] == i) & (df_all['Pclass'] == j + 1)]['Age'].dropna().median()

fare processing

In [173]:
df_all.loc[ df_all['Fare'] <= 7.91, 'Fare'] = 0
df_all.loc[(df_all['Fare'] > 7.91) & (df_all['Fare'] <= 14.454), 'Fare'] = 1
df_all.loc[(df_all['Fare'] > 14.454) & (df_all['Fare'] <= 31), 'Fare']  = 2
df_all.loc[ df_all['Fare'] > 31, 'Fare'] = 3

In [174]:
df_all['Fare'].fillna(df_all['Fare'].dropna().median(), inplace=True)

Is alone processing

In [175]:
df_all['IsAlone'] = 0
df_all.loc[df_all['SibSp'] + df_all['Parch'] == 0, 'IsAlone'] = 1

In [176]:
df_all = df_all.drop(columns=['SibSp', 'Parch'])

Title

In [177]:
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

In [178]:
df_all['Name'] = df_all['Name'].apply(get_title)
df_all['Name'] = df_all['Name'].replace(['Lady', 'Countess','Capt', 'Col',\
'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

df_all['Name'] = df_all['Name'].replace('Mlle', 'Miss')
df_all['Name'] = df_all['Name'].replace('Ms', 'Miss')
df_all['Name'] = df_all['Name'].replace('Mme', 'Mrs')
        

In [179]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
df_all['Name'] = df_all['Name'].map(title_mapping)
df_all['Name'] = df_all['Name'].fillna(0)

Sex

In [180]:
df_all['Sex'] = df_all['Sex'].map({'female': 0, 'male': 1}).astype(int)

Embarked

In [181]:
df_all['Embarked'] = df_all['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

Age*class

In [182]:
df_all['Age*class'] = df_all['Age'] * df_all['Pclass']

In [183]:
df_all.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Fare,Embarked,IsAlone,Age*class
0,1,0.0,3,1,1,1.0,0.0,0,0,3.0
1,2,1.0,1,3,0,2.0,3.0,1,0,2.0
2,3,1.0,3,2,0,1.0,1.0,0,1,3.0
3,4,1.0,1,3,0,2.0,3.0,0,0,2.0
4,5,0.0,3,1,1,2.0,1.0,0,1,6.0


Split df on test and train set's

In [184]:
X_train = df_all.iloc[:num_train].drop(columns=['Survived', 'PassengerId'])
y_train = df_all.iloc[:num_train]['Survived']

In [185]:
X_test = df_all.iloc[num_train:].drop(columns=['Survived'])

In [186]:
y_train.head()

0    0.0
1    1.0
2    1.0
3    1.0
4    0.0
Name: Survived, dtype: float64

In [187]:
ID = X_test['PassengerId']
X_test = X_test.drop(columns=['PassengerId'])

In [188]:
X_test.head()

,Pclass,Name,Sex,Age,Fare,Embarked,IsAlone,Age*class
0,3,1,1,2.0,0.0,2,1,6.0
1,3,3,0,2.0,0.0,0,0,6.0
2,2,1,1,3.0,1.0,2,1,6.0
3,3,1,1,1.0,1.0,0,1,3.0
4,3,3,0,1.0,1.0,0,0,3.0


# training

In [195]:
def create_model(layers=1, neurons=20):
    model = Sequential()
    model.add(InputLayer(input_shape=(X_train.shape[1],)))
    
    for _ in range(layers):
        model.add(Dense(neurons, activation='tanh'))
    
    model.add(Dense(1, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    return model

In [196]:
model = KerasRegressor(build_fn=create_model, epochs=100, verbose=0)

In [197]:
gsc = GridSearchCV(
    estimator=model,
    param_grid={
        'neurons': range(20, 100, 10),
        'layers': range(1, 3)
    },
    scoring='f1',
    cv=3
)

In [198]:
grid_result = gsc.fit(X_train, y_train)

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
"Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)